In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [41]:
df=pd.read_csv('C:\\Users\\Om\Documents\\PyPractice\\UCI_Credit_Card.csv',index_col='ID')
df.rename(columns=lambda x:x.lower(),inplace=True)
df.head()

,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
3,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
4,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [42]:
df['grad_school']=(df['education']==1).astype('int')
df['university']=(df['education']==2).astype('int')
df['high school']=(df['education']==3).astype('int')
df.drop('education',axis=1,inplace=True)
df['male']=(df['sex']==1).astype('int')
df.drop('sex',axis=1,inplace=True)
df['married']=(df['marriage']==1).astype('int')
df.head()


,limit_bal,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,...,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default.payment.next.month,grad_school,university,high school,male,married
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,1,24,2,2,-1,-1,-2,-2,3913.0,...,0.0,0.0,0.0,0.0,1,0,1,0,0,1
2,120000.0,2,26,-1,2,0,0,0,2,2682.0,...,1000.0,1000.0,0.0,2000.0,1,0,1,0,0,0
3,90000.0,2,34,0,0,0,0,0,0,29239.0,...,1000.0,1000.0,1000.0,5000.0,0,0,1,0,0,0
4,50000.0,1,37,0,0,0,0,0,0,46990.0,...,1200.0,1100.0,1069.0,1000.0,0,0,1,0,0,1
5,50000.0,1,57,-1,0,-1,0,0,0,8617.0,...,10000.0,9000.0,689.0,679.0,0,0,1,0,1,1


In [53]:
pay_features=['pay_0','pay_2','pay_3','pay_4','pay_5','pay_6']
for p in pay_features:
    df.loc[df[p]<=0,p]=0
df.rename(columns={'default.payment.next.month':'default'},inplace=True)
df.head()

,limit_bal,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,...,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default,grad_school,university,high school,male,married
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,1,24,2,2,0,0,0,0,3913.0,...,0.0,0.0,0.0,0.0,1,0,1,0,0,1
2,120000.0,2,26,0,2,0,0,0,2,2682.0,...,1000.0,1000.0,0.0,2000.0,1,0,1,0,0,0
3,90000.0,2,34,0,0,0,0,0,0,29239.0,...,1000.0,1000.0,1000.0,5000.0,0,0,1,0,0,0
4,50000.0,1,37,0,0,0,0,0,0,46990.0,...,1200.0,1100.0,1069.0,1000.0,0,0,1,0,0,1
5,50000.0,1,57,0,0,0,0,0,0,8617.0,...,10000.0,9000.0,689.0,679.0,0,0,1,0,1,1


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,precision_recall_curve
from sklearn.preprocessing import RobustScaler

In [69]:
target_name='default'
X=df.drop('default',axis=1)
X.head()
robust_scaler=RobustScaler()
X=robust_scaler.fit_transform(X)
y=df[target_name]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=123,stratify=y)



In [78]:
def CMatrix(CM,labels=['pay','default']):
    df_CM=pd.DataFrame(data=CM,index=labels,columns=labels)
    df_CM.index.name='TRUE'
    df_CM.columns.name='PREDICTION'
    df_CM.loc['Total']=df_CM.sum()
    df_CM['Total']=df_CM.sum(axis=1)
    return df_CM
    
    

In [98]:
metrics=pd.DataFrame(index=['accuracy','precision','recall'],columns=['NULL','LogisticReg','NaiveBayes'])

In [99]:
#NullModel:always predict the most common category
y_pred_test=np.repeat(y_train.value_counts().idxmax(),y_test.size)
metrics.loc['accuracy','NULL']=accuracy_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['precision','NULL']=precision_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['recall','NULL']=recall_score(y_pred=y_pred_test,y_true=y_test)

CM=confusion_matrix(y_pred=y_pred_test,y_true=y_test)
CMatrix(CM)

C:\Program Files\Anaconda2\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


PREDICTION,pay,default,Total
TRUE,,,
pay,3505,0,3505
default,995,0,995
Total,4500,0,4500


In [100]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression


logistic_regression=LogisticRegression(n_jobs=-1,random_state=15)
logistic_regression.fit(X_train,y_train)

y_pred_test=logistic_regression.predict(X_test)
metrics.loc['accuracy','LogisticReg']=accuracy_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['precision','LogisticReg']=precision_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['recall','LogisticReg']=recall_score(y_pred=y_pred_test,y_true=y_test)
CM=confusion_matrix(y_pred=y_pred_test,y_true=y_test)
CMatrix(CM)


C:\Program Files\Anaconda2\envs\py35\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


PREDICTION,pay,default,Total
TRUE,,,
pay,3366,139,3505
default,671,324,995
Total,4037,463,4500


In [101]:
from sklearn.naive_bayes import GaussianNB
NBC=GaussianNB()
NBC.fit(X_train,y_train)

y_pred_test=NBC.predict(X_test)
metrics.loc['accuracy','NaiveBayes']=accuracy_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['precision','NaiveBayes']=precision_score(y_pred=y_pred_test,y_true=y_test)
metrics.loc['recall','NaiveBayes']=recall_score(y_pred=y_pred_test,y_true=y_test)

CM=confusion_matrix(y_pred=y_pred_test,y_true=y_test)
CMatrix(CM)

    

PREDICTION,pay,default,Total
TRUE,,,
pay,2913,592,3505
default,440,555,995
Total,3353,1147,4500


In [102]:
100*metrics

,NULL,LogisticReg,NaiveBayes
accuracy,77.8889,82,77.0667
precision,0,69.9784,48.3871
recall,0,32.5628,55.7789
